# Stock Options Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import yfinance as yf
sns.set_style("whitegrid")

## Data

Let us try to study the Black-Scholes Model, which require the information

i. ***Stock Price (S)***: the corresponding stock prices for each option.\

ii.***Strike Price (K)***: data under the column strike_price.

iii. ***Time to Expiry (T)***: subtracting the current date from the expiration date (exdate). Convert the resulting timedelta to years.

iv. ***Risk-free Interest Rate (r)***: determine an appropriate risk-free interest rate to use in the model. 

v. ***Volatility (σ)***: implied volatility, available in your data. 

### Load options data and download stock data

In [15]:
current_dir = os.getcwd()
# print("Current directory:", current_dir)

# Read the CSV file into a DataFrame
options_data = pd.read_csv('../data/AMD_options_data.csv')

# Check if stock data file exists
stock_data_file = '../data/AMD_stock_data.csv'
if os.path.exists(stock_data_file):
    # If file exists, load it directly
    stock_data = pd.read_csv(stock_data_file)
else:
    # Download stock data
    ticker_symbol = 'AMD'
    start_date = options_data['date'].min()
    end_date = (pd.to_datetime(options_data['date'].max()) + pd.Timedelta(days=1)).date()

    stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

    # Save stock data
    stock_data.to_csv(stock_data_file, index=True)

print(options_data.info())
print(stock_data.info())

/var/folders/q5/69ygwl597nd8rn5pmqr01b580000gn/T/ipykernel_10943/2684269079.py:5: DtypeWarning: Columns (23,26,32) have mixed types. Specify dtype option on import or set low_memory=False.
  options_data = pd.read_csv('../data/AMD_options_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 38 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   secid             1386118 non-null  int64  
 1   date              1386118 non-null  object 
 2   symbol            1386118 non-null  object 
 3   symbol_flag       1386118 non-null  int64  
 4   exdate            1386118 non-null  object 
 5   last_date         1263749 non-null  object 
 6   cp_flag           1386118 non-null  object 
 7   strike_price      1386118 non-null  int64  
 8   best_bid          1386118 non-null  float64
 9   best_offer        1386118 non-null  float64
 10  volume            1386118 non-null  int64  
 11  open_interest     1386118 non-null  int64  
 12  impl_volatility   1166644 non-null  float64
 13  delta             1166644 non-null  float64
 14  gamma             1166644 non-null  float64
 15  vega              1166644 non-null  float64
 16  

### Marge data

In [16]:

# Select relevant columns
selected_columns = ['date', 'exdate', 'cp_flag', 'strike_price', 'best_bid', 'best_offer', 
                    'impl_volatility', 'delta', 'gamma', 'vega', 'theta']

options_data = options_data[selected_columns].copy()
options_data['date'] = pd.to_datetime(options_data['date']).dt.date
print(options_data.info())

stock_data = stock_data[['Date', 'Adj Close', 'Volume']].copy()
stock_data = stock_data.rename(columns={'Date':'date'})
stock_data['date'] = pd.to_datetime(stock_data['date']).dt.date
print(stock_data.info())

merged_data_file = '../data/AMD_merged_data.csv'
if os.path.exists(merged_data_file):
    merged_data = pd.read_csv(merged_data_file)
else:
    # Merge datasets based on date using inner join
    merged_data = pd.merge(options_data, stock_data, on='date')
    merged_data.to_csv('../data/AMD_merged_data.csv', index=False)


print(merged_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 11 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   date             1386118 non-null  object 
 1   exdate           1386118 non-null  object 
 2   cp_flag          1386118 non-null  object 
 3   strike_price     1386118 non-null  int64  
 4   best_bid         1386118 non-null  float64
 5   best_offer       1386118 non-null  float64
 6   impl_volatility  1166644 non-null  float64
 7   delta            1166644 non-null  float64
 8   gamma            1166644 non-null  float64
 9   vega             1166644 non-null  float64
 10  theta            1166644 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 116.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date 

In [17]:
print(merged_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   date             1386118 non-null  object 
 1   exdate           1386118 non-null  object 
 2   cp_flag          1386118 non-null  object 
 3   strike_price     1386118 non-null  int64  
 4   best_bid         1386118 non-null  float64
 5   best_offer       1386118 non-null  float64
 6   impl_volatility  1166644 non-null  float64
 7   delta            1166644 non-null  float64
 8   gamma            1166644 non-null  float64
 9   vega             1166644 non-null  float64
 10  theta            1166644 non-null  float64
 11  Adj Close        1386118 non-null  float64
 12  Volume           1386118 non-null  int64  
dtypes: float64(8), int64(2), object(3)
memory usage: 137.5+ MB
None


### Clean data

In [18]:
data = merged_data.dropna().copy()

# Convert date columns to datetime
data['date'] = pd.to_datetime(data['date'])
data['exdate'] = pd.to_datetime(data['exdate'])


# Example of outlier removal for numeric columns (you can adjust the threshold based on your data)
numeric_columns = ['strike_price', 'best_bid', 'best_offer', 'impl_volatility', 'delta', 'gamma', 'vega', 'theta', 'Adj Close', 'Volume']
for col in numeric_columns:
    Q1 = merged_data[col].quantile(0.25)
    Q3 = merged_data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_data = merged_data[(merged_data[col] >= lower_bound) & (merged_data[col] <= upper_bound)]

# Check the cleaned data
print(outlier_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1362240 entries, 0 to 1386117
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   date             1362240 non-null  object 
 1   exdate           1362240 non-null  object 
 2   cp_flag          1362240 non-null  object 
 3   strike_price     1362240 non-null  int64  
 4   best_bid         1362240 non-null  float64
 5   best_offer       1362240 non-null  float64
 6   impl_volatility  1146412 non-null  float64
 7   delta            1146412 non-null  float64
 8   gamma            1146412 non-null  float64
 9   vega             1146412 non-null  float64
 10  theta            1146412 non-null  float64
 11  Adj Close        1362240 non-null  float64
 12  Volume           1362240 non-null  int64  
dtypes: float64(8), int64(2), object(3)
memory usage: 145.5+ MB
None


In [19]:
outlier_data.describe()

,strike_price,best_bid,best_offer,impl_volatility,delta,gamma,vega,theta,Adj Close,Volume
count,1.362240e+06,1.362240e+06,1.362240e+06,1.146412e+06,1.146412e+06,1.146412e+06,1.146412e+06,1.146412e+06,1.362240e+06,1.362240e+06
mean,1.638915e+05,3.687146e+01,3.825431e+01,6.528334e-01,-9.500788e-03,9.288305e-03,1.646931e+01,-1.883926e+01,8.479102e+01,8.493306e+07
std,1.095931e+05,5.991825e+01,6.112931e+01,3.339461e-01,5.884128e-01,1.368124e-02,2.152126e+01,3.205152e+01,1.642251e+01,2.611119e+07
min,5.000000e+03,0.000000e+00,1.000000e-02,8.018000e-02,-9.999280e-01,0.000000e+00,5.160000e-04,-1.014567e+03,5.594000e+01,2.601890e+07
25%,8.500000e+04,8.500000e-01,1.190000e+00,4.811200e-01,-4.265110e-01,1.694000e-03,2.104176e+00,-2.116417e+01,7.245000e+01,6.655780e+07
50%,1.300000e+05,1.095000e+01,1.175000e+01,5.558010e-01,1.512000e-03,4.522000e-03,7.876283e+00,-9.864411e+00,8.464000e+01,8.349360e+07
75%,2.150000e+05,4.615000e+01,4.845000e+01,6.948887e-01,3.884873e-01,1.094500e-02,2.216930e+01,-3.971274e+00,9.693000e+01,1.006094e+08
max,6.000000e+05,4.853000e+02,4.915500e+02,2.999406e+00,9.999960e-01,3.568210e-01,1.536658e+02,8.139670e+00,1.233400e+02,1.531298e+08


In [20]:
data.describe()

,date,exdate,strike_price,best_bid,best_offer,impl_volatility,delta,gamma,vega,theta,Adj Close,Volume
count,1166644,1166644,1.166644e+06,1.166644e+06,1.166644e+06,1.166644e+06,1.166644e+06,1.166644e+06,1.166644e+06,1.166644e+06,1.166644e+06,1.166644e+06
mean,2022-08-19 11:53:22.749768448,2023-02-01 21:27:38.430850816,1.590629e+05,2.972666e+01,3.089889e+01,6.532276e-01,-1.093080e-02,9.271234e-03,1.646618e+01,-1.884714e+01,8.475904e+01,8.639347e+07
min,2022-02-28 00:00:00,2022-03-04 00:00:00,5.000000e+03,0.000000e+00,1.000000e-02,8.018000e-02,-9.999280e-01,0.000000e+00,5.160000e-04,-1.014567e+03,5.594000e+01,2.601890e+07
25%,2022-05-18 00:00:00,2022-08-26 00:00:00,8.300000e+04,6.300000e-01,8.000000e-01,4.814620e-01,-4.304412e-01,1.695000e-03,2.109112e+00,-2.116794e+01,7.245000e+01,6.655780e+07
50%,2022-08-15 00:00:00,2022-12-30 00:00:00,1.250000e+05,7.850000e+00,8.400000e+00,5.562720e-01,1.490000e-03,4.514000e-03,7.883527e+00,-9.868769e+00,8.464000e+01,8.380670e+07
75%,2022-11-16 00:00:00,2023-04-21 00:00:00,2.050000e+05,3.505000e+01,3.680000e+01,6.954575e-01,3.862302e-01,1.091800e-02,2.219063e+01,-3.972248e+00,9.693000e+01,1.016726e+08
max,2023-02-28 00:00:00,2025-06-20 00:00:00,6.000000e+05,4.825000e+02,4.865500e+02,2.999406e+00,9.999960e-01,3.568210e-01,1.536658e+02,8.139670e+00,1.233400e+02,2.253941e+08
std,NaN,NaN,1.073532e+05,5.161338e+01,5.285049e+01,3.339506e-01,5.886337e-01,1.365108e-02,2.148920e+01,3.203529e+01,1.635851e+01,2.900568e+07


In [6]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386118 entries, 0 to 1386117
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   date             1386118 non-null  datetime64[ns]
 1   exdate           1386118 non-null  datetime64[ns]
 2   cp_flag          1386118 non-null  object        
 3   strike_price     1386118 non-null  int64         
 4   best_bid         1386118 non-null  float64       
 5   best_offer       1386118 non-null  float64       
 6   impl_volatility  1166644 non-null  float64       
 7   delta            1166644 non-null  float64       
 8   gamma            1166644 non-null  float64       
 9   vega             1166644 non-null  float64       
 10  theta            1166644 non-null  float64       
 11  Adj Close        1386118 non-null  float64       
 12  Volume           1386118 non-null  int64         
dtypes: datetime64[ns](2), float64(8), int64(2), object(1)
mem

In [22]:
data.head()

,date,exdate,cp_flag,strike_price,best_bid,best_offer,impl_volatility,delta,gamma,vega,theta,Adj Close,Volume,dte,moneyness
0,2022-02-28,2022-03-04,P,143000,18.30,21.70,0.919996,-0.931618,0.011129,1.700479,-71.04977,123.339996,124666100,4,0.000863
1,2022-02-28,2022-03-04,P,144000,19.65,22.55,1.009123,-0.921234,0.011298,1.898582,-86.91409,123.339996,124666100,4,0.000857
2,2022-02-28,2022-03-04,P,145000,20.45,23.70,1.029200,-0.926204,0.010543,1.805097,-84.34332,123.339996,124666100,4,0.000851
3,2022-02-28,2022-03-04,P,146000,21.30,24.75,1.032943,-0.933979,0.009645,1.653858,-77.66636,123.339996,124666100,4,0.000845
4,2022-02-28,2022-03-04,P,147000,23.50,25.75,1.347105,-0.879729,0.011528,2.587893,-158.53520,123.339996,124666100,4,0.000839


In [21]:

# Step 3: Feature Engineering, compute days to expiration
data['dte'] = (data['exdate'] - data['date']).dt.days
data['moneyness'] = data['Adj Close'] / data['strike_price']



In [ ]:
# Convert date column to datetime if not already in datetime format
merged_data['date'] = pd.to_datetime(merged_data['date'])

# Filter rows for the end of February 28, 2022, and the beginning of March 1, 2022
filtered_rows = merged_data[(merged_data['date'] == '2022-02-28 23:59:59') | (merged_data['date'] == '2022-03-01 00:00:00')]

print(filtered_rows)


In [8]:
import yfinance as yf

# Define the stock symbol for which you want to fetch options data
stock_symbol = 'AMD'  # Example: Apple Inc.

# Create a Ticker object for the stock symbol
ticker = yf.Ticker(stock_symbol)

# Get options data
options_data = ticker.options

# Print the available expiration dates for options
print("Available expiration dates for options:")
print(options_data)


Available expiration dates for options:
('2024-04-05', '2024-04-12', '2024-04-19', '2024-04-26', '2024-05-03', '2024-05-10', '2024-05-17', '2024-05-24', '2024-06-21', '2024-07-19', '2024-08-16', '2024-09-20', '2024-10-18', '2024-11-15', '2024-12-20', '2025-01-17', '2025-03-21', '2025-06-20', '2025-08-15', '2025-10-17', '2025-11-21', '2025-12-19', '2026-01-16', '2026-06-18', '2026-12-18')


In [ ]:
df.info()

In [ ]:
df2.info()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
import yfinance as yf

data = yf.download('AMD', start=df['date'].min(), end=df['date'].max())
# Print data
data

In [ ]:
df['date'].min()

In [ ]:
df['date'].max()

In [ ]:
import math

import numpy as np
import QuantLib as ql


In [ ]:
import sys
print(sys.path)


In [ ]:
import pkg_resources

# Get a list of all installed packages
installed_packages = pkg_resources.working_set

# Print each package name
for package in installed_packages:
    print(package)


In [ ]:
import sys

# Print out all packages in the current Anaconda environment
for line in sys.path:
    print(line)
